## Pt 1

In [109]:
data class Ruleset(val key: Char, val cmp: Char, val n: Int, val destination: String)

val (block1, block2) = File("input.txt").readText().split("\n\n")
val workflows = hashMapOf<String, Pair<List<Ruleset>, String>>()

block1.lines().forEach { line ->
    val (name, rest) = line.dropLast(1).split("{")
    workflows[name] = Pair(rest.split(",").dropLast(1).map {
        val (c, g) = it.split(":")
        Ruleset(c.first(), c[1], c.substring(2).toInt(), g)
    }, rest.substringAfterLast(",").split(":").last())
}

fun accept(item: Map<Char, Int>, name: String = "in"): Boolean {
    val ops = mapOf(
        '>' to { a: Int, b: Int -> a > b },
        '<' to { a: Int, b: Int -> a < b }
    )

    if (name == "R") return false
    if (name == "A") return true
    val (rules, fallback) = workflows[name]!!
    for ((key, cmp, n, target) in rules) {
        if (ops[cmp]!!(item[key]!!, n)) return accept(item, target)
    }
    return accept(item, fallback)
}

block2.lines().fold(0) { acc, line ->
    line.substring(1, line.length - 1).split(",").associate {
        val (ch, n) = it.split("=")
        ch[0] to n.toInt()
    }.let { if (accept(it)) acc + it.values.sum() else acc + 0 }
}


432788

## Pt 2

In [110]:
data class Ruleset(val key: Char, val cmp: Char, val n: Int, val destination: String)

val (block1, _) = File("input.txt").readText().split("\n\n")
val workflows = hashMapOf<String, Pair<List<Ruleset>, String>>()

block1.lines().forEach { line ->
    val (name, rest) = line.dropLast(1).split("{")
    workflows[name] = Pair(rest.split(",").dropLast(1).map {
        val (c, g) = it.split(":")
        Ruleset(c.first(), c[1], c.substring(2).toInt(), g)
    }, rest.substringAfterLast(",").split(":").last())
}

fun count(ranges: Map<Char, IntRange>, name: String = "in"): Long {
    if (name == "R") return 0
    if (name == "A") return ranges.values.fold(1L) { acc, range -> acc * (range.last - range.first + 1) }
    val (rules, fallback) = workflows[name]!!
    var total = 0L
    val nr = ranges.toMutableMap()
    for ((key, cmp, n, target) in rules) {
        val range = nr[key]!!
        val (a, r) = if (cmp == '>') n + 1..range.last to range.first..n else range.first..<n to n..range.last
        if (a.first <= a.last) total += count(nr.apply { put(key, a) }, target)
        if (r.first <= r.last) nr[key] = r else break
    }
    return total + count(nr, fallback)
}
count("xmas".associate { it to (1..4000) })

142863718918201